In [ ]:
# load_overlay_and_test_conv1.py
import numpy as np
from pynq import Overlay, allocate
from time import sleep
from PIL import Image

# CNN dimensions
IMG_H, IMG_W, IMG_C = 28, 28, 1
OUT_H, OUT_W, OUT_C = 13, 13, 32

# 1) Load overlay and get IP handle
ol = Overlay('asl_system.bit', download=True)
asl_conv1 = ol.asl_conv1_0

print("Overlay loaded. IPs:", ol.ip_dict.keys())
print("asl_conv1 register map before run:")
print(asl_conv1.register_map)

# 2) Allocate input/output buffers
inp  = allocate(shape=(IMG_H, IMG_W, IMG_C), dtype=np.float32)
outp = allocate(shape=(OUT_H, OUT_W, OUT_C), dtype=np.float32)

# Option A: random test pattern
inp[:] = np.random.rand(IMG_H, IMG_W, IMG_C).astype(np.float32)

# Option B: real ASL test image (uncomment if you have the file)
# img = Image.open('test_asl_image.png').convert('L')
# img = img.resize((IMG_W, IMG_H))
# img_np = np.array(img, dtype=np.float32) / 255.0
# img_np = img_np.reshape(IMG_H, IMG_W, IMG_C)
# inp[:] = img_np

inp.flush()

# 3) Program accelerator registers and start
reg = asl_conv1.register_map
reg.in_r_1  = inp.device_address
reg.out_r_1 = outp.device_address
reg.CTRL.AP_START = 1

# 4) Bounded polling loop for AP_DONE
for i in range(1000000):
    if reg.CTRL.AP_DONE == 1:
        break
    sleep(0.0001)

ctrl = reg.CTRL
print("CTRL register after run:", ctrl)

# 5) Read and print feature map
if ctrl.AP_DONE == 1:
    outp.invalidate()
    feat_hw = np.array(outp)
    print("HW feature map shape:", feat_hw.shape)
    print("First 16 values:", feat_hw.ravel()[:16])
    np.save('feat_hw_conv1_test.npy', feat_hw.astype(np.float32))
else:
    print("ERROR: asl_conv1 did not complete (AP_DONE=0).")
